In [2]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import IPython.display
import scipy
import pandas as pd

In [22]:
data_nc, sr = librosa.load('data/c_nc_1.wav', sr =  None, mono = True)
data_c, sr = librosa.load('data/c_c_1.wav', sr =  None, mono = True)
print(sr)

n_sec = 15

data_nc = data_nc[:sr * n_sec]
data_c = data_c[:sr * n_sec]
n = min(len(data_nc), len(data_c))

16000


In [19]:
def lowpass(data, sr, cutoff=1000):
    b, a = scipy.signal.butter(6, cutoff, btype='low', fs=sr)
    return scipy.signal.lfilter(b, a, data)

data_nc = lowpass(data_nc, sr)
data_c = lowpass(data_c, sr)

In [20]:
IPython.display.Audio(data=data_nc, rate=sr)

In [21]:
IPython.display.Audio(data=data_c, rate=sr)

In [37]:
samples_per_segment = (sr // 1000) * 30 # segment = 30 miliseconds
n = samples_per_segment * (n // samples_per_segment)

df = pd.DataFrame(np.concatenate((data_nc[:n], data_c[:n])))
df['id'] = np.arange(n * 2) // samples_per_segment
df['time'] = np.tile(np.arange(samples_per_segment), n // samples_per_segment * 2)

df

,0,id,time
0,0.016556,0,0
1,0.016968,0,1
2,0.017426,0,2
3,0.017883,0,3
4,0.018372,0,4
...,...,...,...
319675,-0.003860,665,475
319676,-0.003952,665,476
319677,-0.004135,665,477
319678,-0.004440,665,478


In [40]:
labels = [0] * (n // samples_per_segment)
labels += [1] * (n // samples_per_segment)

labels = pd.Series(labels)
labels

0      0
1      0
2      0
3      0
4      0
      ..
661    1
662    1
663    1
664    1
665    1
Length: 666, dtype: int64

In [41]:
import tsfresh

feats = tsfresh.extract_relevant_features(df, labels, column_id='id', column_sort='time')
kind_to_fc_parameters = tsfresh.feature_extraction.settings.from_columns(feats)

print(kind_to_fc_parameters)

Feature Extraction: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [01:23<00:00,  4.17s/it]


{'0': {'number_peaks': [{'n': 10}, {'n': 5}, {'n': 3}, {'n': 1}], 'augmented_dickey_fuller': [{'attr': 'usedlag', 'autolag': 'AIC'}], 'spkt_welch_density': [{'coeff': 5}], 'ar_coefficient': [{'coeff': 10, 'k': 10}, {'coeff': 4, 'k': 10}, {'coeff': 3, 'k': 10}, {'coeff': 2, 'k': 10}, {'coeff': 5, 'k': 10}, {'coeff': 6, 'k': 10}, {'coeff': 1, 'k': 10}, {'coeff': 9, 'k': 10}], 'sample_entropy': None, 'approximate_entropy': [{'m': 2, 'r': 0.1}, {'m': 2, 'r': 0.3}, {'m': 2, 'r': 0.5}, {'m': 2, 'r': 0.7}, {'m': 2, 'r': 0.9}], 'matrix_profile': [{'feature': 'min', 'threshold': 0.98}, {'feature': 'max', 'threshold': 0.98}, {'feature': 'mean', 'threshold': 0.98}, {'feature': '25', 'threshold': 0.98}], 'permutation_entropy': [{'dimension': 7, 'tau': 1}, {'dimension': 3, 'tau': 1}, {'dimension': 6, 'tau': 1}, {'dimension': 5, 'tau': 1}, {'dimension': 4, 'tau': 1}], 'cid_ce': [{'normalize': True}], 'fourier_entropy': [{'bins': 100}, {'bins': 10}, {'bins': 3}], 'fft_coefficient': [{'attr': 'abs', '